In [3]:
import pandas as pd
import torch
import numpy as np
from transformers import BertModel, BertTokenizer

In [4]:
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
df=pd.read_csv('sentiment_tweets - Copy.csv')

In [6]:
# Define batch size
batch_size = 32

In [8]:
embeddings = []
for i in range(0, len(df), batch_size):
    batch = df['Tweet_Content'][i:i+batch_size].tolist()
    tokens = tokenizer.batch_encode_plus(
        batch, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**tokens)
    batch_embeddings = outputs[0][:, 0, :].numpy()
    embeddings.append(batch_embeddings)

In [9]:
# Concatenate the embeddings and convert to a pandas DataFrame
embeddings = np.concatenate(embeddings, axis=0)
df_emb = pd.DataFrame(embeddings)


In [10]:
# Concatenate the original tweets DataFrame and the embeddings DataFrame
df_final = pd.concat([df, df_emb], axis=1)

In [11]:
# Save the DataFrame to a CSV file
df_final.to_csv('tweet_embeddings.csv', index=False)
